In [22]:
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
local_base_url = 'http://localhost:2000'

In [23]:
import os
import sys

backend_path = os.path.abspath(os.path.join(os.getcwd(), '../backend'))
sys.path.append(backend_path)

from app.internal.firebase import db

In [ ]:
exchange_token = "100"
smallcase_name = "Amara Raja Energy & Mobility Ltd"

start_date = "2023-12-15"
end_date = "2024-03-14"

q = await db.collection("tickers").where("smallcase_name", "==", smallcase_name).get()

In [9]:
if q:
    equity = q[0].to_dict()
    resp = requests.get(f"{local_base_url}/tickers/{equity['exchange_token']}/candles?start_date={start_date}&end_date={end_date}")
    print(resp.status_code)

200


In [27]:
candles: dict = resp.json()

In [36]:
df = pd.DataFrame.from_dict(candles, orient="index")
df.index.name = "date"
df.head()

,open,high,low,close,volume,open_interest
date,,,,,,
2023-12-15,774.50,779.90,763.05,765.25,352696,0
2023-12-18,768.90,789.45,757.15,781.25,965918,0
2023-12-19,789.95,799.60,778.25,781.95,846627,0
2023-12-20,784.00,786.50,737.80,743.80,935363,0
2023-12-21,731.55,765.25,728.50,762.45,661832,0


In [37]:
df.to_csv("candles.csv")

In [42]:
import math
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [72]:
dates = list(candles.keys())
n = len(dates)

returns = []
for i in range(n - 1):
    prev_price = candles[dates[i]]["close"]
    next_price = candles[dates[i + 1]]["close"]
    r = math.log(prev_price / next_price)
    returns.append(r)

s = sum(returns)
n -= 1 # Adjusting for the last data point
mean = s / n
print("Mean:", mean)

q = 0
for r in returns:
    q += (r - mean) ** 2

print("Sum of squared deviations:", q)
print("Variance:", q / n)

variance = np.var(returns)
print("Variance:", variance)
print("SD:", variance ** 0.5)

print(s, n, s/n)



Mean: -0.0006940839300399
Sum of squared deviations: 0.02625307663087031
Variance: 0.0004234367198527469
Variance: 0.00042343671985274703
SD: 0.02057757808520592
-0.0430332036624738 62 -0.0006940839300399


### Backtesting

In [145]:
DATE_FORMAT = "%Y-%m-%d"
smallcase_id = "SCAW_0001"
quarter_end = "2019-01-01"
params = {"date": quarter_end}

smallcase = requests.get(f"{local_base_url}/smallcases/{smallcase_id}").json()
constituents = requests.get(f"{local_base_url}/smallcases/{smallcase_id}/constituents", params=params).json()

start_date = constituents["start_date"]
end_date = constituents["end_date"]

# prev_start_date = datetime.strptime(start_date, DATE_FORMAT) - relativedelta(months=3)
# prev_start_date = prev_start_date.strftime(DATE_FORMAT)

# prev_end_date = datetime.strptime(end_date, DATE_FORMAT) - relativedelta(months=3)
# prev_end_date = prev_end_date.strftime(DATE_FORMAT)

In [146]:
async def get_candles_from_smallcase_name(smallcase_name: str, start_date, end_date):
    q = await db.collection("tickers").where("smallcase_name", "==", smallcase_name).get()
    if q:
        equity = q[0].to_dict()
        resp = requests.get(f"{local_base_url}/tickers/{equity['exchange_token']}/candles?start_date={start_date}&end_date={end_date}")
        if resp.status_code == 200:
            return resp.json()
        print(resp.status_code, "Not Found")

In [147]:
async def get_candles(constituents, start_date, end_date):
    candles = {}
    for c in constituents:
        # Maybe BE can have a call for this
        ticker = await db.collection("tickers").where("smallcase_name", "==", c["smallcase_name"]).get()
        if ticker:
            ticker = ticker[0].to_dict()
            params = {"start_date": start_date, "end_date": end_date}
            resp = requests.get(f"{local_base_url}/tickers/{ticker['exchange_token']}/candles", params=params)
            candles[c["smallcase_name"]] = resp.json()

    return candles

In [148]:
c = constituents["constituents"]
candles = await get_candles(c, start_date, end_date)
len(candles) == len(c)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:290: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
/Users/aaryanpagar/Library/Python/3.11/lib/python/site-packages/google/api_core/retry/retry_unary.py:146: UserWarning: Using the synchronous google.api_core.retry.Retry with asynchronous calls may lead to unexpected results. Please use google.api_core.retry_async.AsyncRetry instead.
  warnings.warn(_ASYNC_RETRY_WARNING)


True

In [158]:
candles_at_inception = {}
inception_date = "2018-07-18"

for constituent in c:
    ticker = await db.collection("tickers").where("smallcase_name", "==", constituent["smallcase_name"]).get()
    if ticker:
        ticker = ticker[0].to_dict()
        resp = requests.get(f"{local_base_url}/tickers/{ticker['exchange_token']}/candles/{inception_date}")
        candles_at_inception[constituent["smallcase_name"]] = resp.json()

len(candles_at_inception) == len(c)

True

In [159]:
def calculate_index(constituents: list, inception_date: str, date: str) -> int:
    index = 100
    indexes = 0
    for c in constituents:
        try:
            name = c["smallcase_name"]
            weight = c["original_weightage"]

            close = candles[name][date]["close"]
            close_at_inception = candles_at_inception[name]["daily"][inception_date]["close"]

            investable_shares = (index * weight) / close_at_inception
            indexes += investable_shares * close
            
        except KeyError as e:
            print(e)
            print(f"Candle not found for {name}. Skipping index on {date}.")
            continue
    return indexes


In [160]:
indexes = calculate_index(c, inception_date, "2019-03-27")
indexes

103.66443091913047

## Expected Value